# Data Analysis of the DSAD

In [9]:
from pycocotools.coco import COCO
import os

In [10]:
# First obtain the json files for each training split

root = r"C:\Users\jayan\Documents\MECHATRONICS YR4\MECH5845M - Professional Project\DSAD4DeTr_multilabel_OD"
subsets = ['val', 'test', 'train']

In [11]:
# Create COCO class instantiation function
def createCOCO(root_path, mode):
    filename = f"{mode}_OD_annotations.json"
    annFile = os.path.join(root_path, mode, 'annotations', filename)
    coco = COCO(annFile)

    return coco

In [22]:
from pprint import pprint

# Define structure for stat storage
base_dict = {
    "bbox": 0,
    "segm": 0,
    "instances": 0,
    "images": 0
}

no_classes = 7
class_dict = {1: {'bbox': 0, 'images': 0, 'instances': 0, 'segm': 0},
    2: {'bbox': 0, 'images': 0, 'instances': 0, 'segm': 0},
    3: {'bbox': 0, 'images': 0, 'instances': 0, 'segm': 0},
    4: {'bbox': 0, 'images': 0, 'instances': 0, 'segm': 0},
    5: {'bbox': 0, 'images': 0, 'instances': 0, 'segm': 0},
    6: {'bbox': 0, 'images': 0, 'instances': 0, 'segm': 0},
    7: {'bbox': 0, 'images': 0, 'instances': 0, 'segm': 0}
    }
# for i in range(no_classes):
#     key = i+1
#     class_dict[key] = base_dict

pprint(class_dict)

{1: {'bbox': 0, 'images': 0, 'instances': 0, 'segm': 0},
 2: {'bbox': 0, 'images': 0, 'instances': 0, 'segm': 0},
 3: {'bbox': 0, 'images': 0, 'instances': 0, 'segm': 0},
 4: {'bbox': 0, 'images': 0, 'instances': 0, 'segm': 0},
 5: {'bbox': 0, 'images': 0, 'instances': 0, 'segm': 0},
 6: {'bbox': 0, 'images': 0, 'instances': 0, 'segm': 0},
 7: {'bbox': 0, 'images': 0, 'instances': 0, 'segm': 0}}


In [18]:
# Test extraction of surgery number
test_id = 21247
test_id_str = str(test_id)
surgery_no = int(test_id_str[1:3])
print(surgery_no)

12


In [43]:
# Fill out dictionary storing stats for each surgery

# Initialise surgery dict
surgery_dict = {}

# 0>bbox, 1>segm, 2>instances, 3>images
class_dict = {
    1: {0: 0, 1: 0, 2: 0, 3: 0},
    2: {0: 0, 1: 0, 2: 0, 3: 0},
    3: {0: 0, 1: 0, 2: 0, 3: 0},
    4: {0: 0, 1: 0, 2: 0, 3: 0},
    5: {0: 0, 1: 0, 2: 0, 3: 0},
    6: {0: 0, 1: 0, 2: 0, 3: 0},
    7: {0: 0, 1: 0, 2: 0, 3: 0}
}

for subset in subsets:
    # Create coco object
    coco = createCOCO(root, subset)
    # Get list of the image IDs present
    imageIDs = coco.getImgIds()

    for image in imageIDs:
        image_str = str(image)
        surgery_no = int(image_str[1:3]) # Get surgery number from image ID
        categories = set() # Reinitialise set for storing the classes present in image

        # Get the annotations for the image
        annIDs = coco.getAnnIds(imgIds=image)
        anns = coco.loadAnns(annIDs)

        for ann in anns:
            # Store the class as being present in this image
            cat_id = ann['category_id']
            categories.add(cat_id)
            
            # Obtain the bbox and segmentation areas
            bbox_area = ann['bbox'][2] * ann['bbox'][3]
            segm_area = ann['area']

            # Create surgery no field in the dictionary if not present already
            if surgery_no not in surgery_dict.keys():
                surgery_dict[surgery_no] = class_dict
            
            # Update fields for the corresponding surgery and class
            surgery_dict[surgery_no][cat_id][0] += bbox_area
            surgery_dict[surgery_no][cat_id][1] += segm_area
            surgery_dict[surgery_no][cat_id][2] += 1
        
        # Update the number of images each class has appeared in
        for cat in categories:
            surgery_dict[surgery_no][cat][3] += 1

# pprint(surgery_dict)

loading annotations into memory...
Done (t=0.40s)
creating index...
index created!
loading annotations into memory...
Done (t=0.53s)
creating index...
index created!
loading annotations into memory...
Done (t=0.90s)
creating index...
index created!


In [46]:
def getSubset(surgery_no):
    """
    Function to obtain the correct subset for each surgery number using
    the split from Kolbinger et al. (2023)
    """
    # Define dictionary lookup with integer keys
    lookup = {
        1: 'train',
        4: 'train',
        5: 'train',
        6: 'train',
        8: 'train',
        9: 'train',
        10: 'train',
        12: 'train',
        15: 'train',
        16: 'train',
        17: 'train',
        19: 'train',
        22: 'train',
        23: 'train',
        24: 'train',
        25: 'train',
        27: 'train',
        28: 'train',
        29: 'train',
        30: 'train',
        31: 'train',
        3: 'val',
        21: 'val',
        26: 'val',
        2: 'test',
        7: 'test',
        11: 'test',
        13: 'test',
        14: 'test',
        18: 'test',
        20: 'test',
        32: 'test'
    }

    return lookup[surgery_no]

In [42]:
import numpy as np

# Convert surgery_dict into a matrix
results = np.zeros((4, 32, 7)) # 4 fields x 32 surgeries x 7 classes

for surgery, classes in surgery_dict.items():
    for class_id, fields in classes.items():
        for field, value in fields.items():
            results[field][surgery-1][class_id-1] = value

# pprint(results)

In [47]:

# Create train, test, val split
split_results = np.zeros((4, 3, 7))

split_mapping = {'train': 0, 'test': 1, 'val': 2}

for y in range(32):
    split = getSubset(y+1)
    output_y = split_mapping[split]
    split_results[:, output_y, :] += results[:, y, :]

print(results)

[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [2.24002314e+08 7.98232420e+07 4.90378670e+07 9.25121900e+06
   9.08261600e+06 8.94470900e+06 1.56941926e+08]
  [2.24002314e+08 7.98232420e+07 4.90378670e+07 9.25121900e+06
   9.08261600e+06 8.94470900e+06 1.56941926e+08]
  [2.24002314e+08 7.98232420e+07 4.90378670e+07 9.25121900e+06
   9.08261600e+06 8.94470900e+06 1.56941926e+08]
  [2.24002314e+08 7.98232420e+07 4.90378670e+07 9.25121900e+06
   9.08261600e+06 8.94470900e+06 1.56941926e+08]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [2.24002314e+08 7.98232420e+07 4.90378670e+07 9.25121900e+06
   9.08261600e+06 8.94470900e+06 1.56941926e+08]
  [2.24002314e+08 7.98232420e+07 4.90378670e+07 9.25121900e+06
   9.08261600e+06 8.94470900e+06 1.56941926e+08]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.000000

In [48]:
# Sum the classes so you have a total for each field in each set
split_results_classes_summed = split_results.sum(axis=2)
pprint(split_results_classes_summed)

array([[8.05625840e+09, 2.68541946e+09, 1.61125168e+09],
       [4.64119838e+09, 1.54706613e+09, 9.28239676e+08],
       [7.96350000e+04, 2.65450000e+04, 1.59270000e+04],
       [5.58600000e+04, 1.86200000e+04, 1.11720000e+04]])


In [50]:
# Calculate the proportion array to store the proportion of each classes fields within each split

# Expand the summed_output_array dimensions to match output_array for broadcasting
expanded_summed_output_array = split_results_classes_summed[:, :, np.newaxis]

# Divide the output_array by the expanded summed_output_array
proportion_array = 100 * split_results / expanded_summed_output_array

print(proportion_array)

[[[41.7071368  14.86234144  9.13039241  1.72249049  1.69109819
    1.6654212  29.22111946]
  [41.7071368  14.86234144  9.13039241  1.72249049  1.69109819
    1.6654212  29.22111946]
  [41.7071368  14.86234144  9.13039241  1.72249049  1.69109819
    1.6654212  29.22111946]]

 [[42.23695522 14.3964053   9.67932316  1.53848773  1.83052809
    1.55430185 28.76399865]
  [42.23695522 14.3964053   9.67932316  1.53848773  1.83052809
    1.55430185 28.76399865]
  [42.23695522 14.3964053   9.67932316  1.53848773  1.83052809
    1.55430185 28.76399865]]

 [[24.95761914 24.82576756  4.95385195  6.95046148  1.80825014
    2.54285176 33.96119797]
  [24.95761914 24.82576756  4.95385195  6.95046148  1.80825014
    2.54285176 33.96119797]
  [24.95761914 24.82576756  4.95385195  6.95046148  1.80825014
    2.54285176 33.96119797]]

 [[25.67132116 19.11922664  6.25671321  5.58539205  2.09452202
    2.87325456 38.39957035]
  [25.67132116 19.11922664  6.25671321  5.58539205  2.09452202
    2.87325456 38.399

I don't think this is correct as all the values are the same in each of the 4 fields